# Federated Learning Model

## Created FL Model


*  3 Layer Conv2D Architecture
*  Data Distribution: Number of agents used = 5
*  Number of Epochs Per Agent = 10
*  Aggregation
*  Global Model


**Imports**:


1.   Tensorflow
2.   Keras
3.   Numpy

In [ ]:
# Authors: Sai Prasad, Purnima Sai Koumudi
# Model saved time: 04/24

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

**Downloading the dataset (CIFAR 10)**

Data Preprossing

In [ ]:
# Loading and normalizing the CIFAR-10 dataset
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    return x_train, y_train, x_test, y_test

**CNN Model Architecture**

Activation: RELU

In [ ]:
# Creating the CNN model architecture
def create_cnn_model():
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10)  # Output layer for 10 classes
    ])
    return model

**Loss used: Sparse Categorical Crossentropy**



In [ ]:
# Function to compile and train the model
def compile_and_train(model, x_train, y_train, epochs=10):
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs)

Defining Federated Learning

1. Num of agents defined = 5
2. Getting Global Model
3. Evaluating the model defined
4. Test Accuracy Calculation

In [ ]:
# Implementing federated learning
def federated_learning(x_train, y_train, num_clients=5):
    x_train_clients = np.array_split(x_train, num_clients)
    y_train_clients = np.array_split(y_train, num_clients)
    global_model = create_cnn_model()

    for x_train, y_train in zip(x_train_clients, y_train_clients):
        local_model = create_cnn_model()
        local_model.set_weights(global_model.get_weights())
        compile_and_train(local_model, x_train, y_train)
        updated_weights = [(g + l) / 2 for g, l in zip(global_model.get_weights(), local_model.get_weights())]
        global_model.set_weights(updated_weights)

    return global_model

x_train, y_train, x_test, y_test = load_and_preprocess_data()
global_model = federated_learning(x_train, y_train)

# Compile the global model before evaluation
global_model.compile(optimizer='adam',
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])

test_loss, test_accuracy = global_model.evaluate(x_test, y_test)
print(f'Test set accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/10
313/313 [==============================] - 31s 97ms/step - loss: 1.8614 - accuracy: 0.2966
Epoch 2/10
313/313 [==============================] - 31s 100ms/step - loss: 1.4800 - accuracy: 0.4613
Epoch 3/10
313/313 [==============================] - 30s 96ms/step - loss: 1.3097 - accuracy: 0.5302
Epoch 4/10
313/313 [==============================] - 30s 96ms/step - loss: 1.1575 - accuracy: 0.5886
Epoch 5/10
313/313 [==============================] - 31s 100ms/step - loss: 1.0650 - accuracy: 0.6223
Epoch 6/10
313/313 [==============================] - 30s 97ms/step - loss: 0.9299 - accuracy: 0.6730
Epoch 7/10
313/313 [==============================] - 30s 96ms/step - loss: 0.8266 - accuracy: 0.7078
Epoch 8/10
313/313 [==============================] - 32s 101ms/step - loss: 0.7101 - accuracy: 0.7469
Epoch 9/10
313/313 [==============================] - 30s 96ms/step - loss: 0.5991 - accuracy: 0.7900
Epoch 10/10
313/313 [==============================] - 31s 99ms/step - loss: 0.

**Model Saving Checkpoint**

In [ ]:
global_model.save("trained_model_2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
